# Chapter 13. 텐서플로를 사용한 신경망 훈련
## 13.1 텐서플로와 훈련 성능
## 13.2 텐서플로 처음 시작하기

In [1]:
import tensorflow as tf
import numpy as np

np.set_printoptions(precision=3)

In [2]:
a = np.array([1, 2, 3], dtype=np.int32)
b = [4, 5, 6]

In [3]:
t_a = tf.convert_to_tensor(a)
t_b = tf.convert_to_tensor(b)


2022-04-21 11:08:37.466700: I tensorflow/core/platform/cpu_feature_guard.cc:145] This TensorFlow binary is optimized with Intel(R) MKL-DNN to use the following CPU instructions in performance critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in non-MKL-DNN operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-21 11:08:37.466898: I tensorflow/core/common_runtime/process_util.cc:115] Creating new thread pool with default inter op setting: 4. Tune using inter_op_parallelism_threads for best performance.


In [ ]:
# Warning message: 
# 최신 CPU들은 AVX, SSE4 같은 명령어들을 제공 - 선형대수 연산을 좀 더 빠르게 수행할 수 있도록 도와주는 역할
# Reference) https://github.com/tensorflow/tensorflow/issues/8037

# Solution 1) 그냥 무시하기 - 속도가 느려도 상관 없는 경우
# import os
# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Solution 2) 환경에 맞게 텐서플로우 바이너리를 다시 컴파일
# Reference) https://www.tensorflow.org/install/source#macos

In [4]:
print(t_a)
print(t_b)

tf.Tensor([1 2 3], shape=(3,), dtype=int32)
tf.Tensor([4 5 6], shape=(3,), dtype=int32)


In [5]:
t_ones = tf.ones((2, 3))
t_ones.shape

TensorShape([2, 3])

In [6]:
t_ones.numpy()

array([[1., 1., 1.],
       [1., 1., 1.]], dtype=float32)

In [7]:
const_tensor = tf.constant([1.2, 5, np.pi], dtype=np.float32)
print(const_tensor)

tf.Tensor([1.2   5.    3.142], shape=(3,), dtype=float32)


### 13.2.3 텐서의 데이터 타입과 크기 조작

In [9]:
# 데이터 타입 바꾸기
t_a_new = tf.cast(t_a, tf.int64)
print(t_a_new.dtype)

<dtype: 'int64'>


In [10]:
# 전치하기 (transpose)
t = tf.random.uniform(shape=(3, 5))
t_tr = tf.transpose(t)
print(t.shape, '-->', t_tr.shape)

(3, 5) --> (5, 3)


In [11]:
# 1차원 배열에서 2차원 배열로
t = tf.zeros((30, ))
t_reshape = tf.reshape(t, shape=(5, 6))
print(t_reshape.shape)

(5, 6)


In [12]:
# 불필요한 차원 삭제하기
t = tf.zeros((1, 2, 1, 4, 1))
t_sqz = tf.squeeze(t, axis=(2, 4))
print(t.shape, '-->', t_sqz.shape)

(1, 2, 1, 4, 1) --> (1, 2, 4)


### 13.2.4 텐서에 수학 연산 적용

In [13]:
tf.random.set_seed(1)
t1 = tf.random.uniform(shape=(5, 2), minval=-1.0, maxval=1.0)
t2 = tf.random.normal(shape=(5, 2), mean=0.0, stddev=1.0)

In [14]:
# 원소별 곱셈
t3 = tf.multiply(t1, t2).numpy()
print(t3)

[[-0.27  -0.874]
 [-0.017 -0.175]
 [-0.296 -0.139]
 [-0.727  0.135]
 [-0.401  0.004]]


In [17]:
# 특정 축을 따라 평균, 합, 표준 편차 계산
t4 = tf.math.reduce_mean(t1, axis=0)  # 열 방향
print(t4)

tf.Tensor([0.09  0.207], shape=(2,), dtype=float32)


In [18]:
# 행렬 곱셈
t5 = tf.linalg.matmul(t1, t2, transpose_b=True)
print(t5.numpy())

[[-1.144  1.115 -0.87  -0.321  0.856]
 [ 0.248 -0.191  0.25  -0.064 -0.331]
 [-0.478  0.407 -0.436  0.022  0.527]
 [ 0.525 -0.234  0.741 -0.593 -1.194]
 [-0.099  0.26   0.125 -0.462 -0.396]]


In [19]:
t6 = tf.linalg.matmul(t1, t2, transpose_a=True)
print(t6.numpy())

[[-1.711  0.302]
 [ 0.371 -1.049]]


In [20]:
# norm 계산하기
norm_t1 = tf.norm(t1, ord=2, axis=1).numpy()
print(norm_t1)

[1.046 0.293 0.504 0.96  0.383]


## 13.3 텐서플로 데이터셋 API (tf.data)를 사용하여 입력 파이프라인 구축

* 매번 전처리 함수를 수동으로 적용하는 것은 매우 번거로우므로 간편한 전처리 파이프라인을 만들어 주는 특별한 클래스를 제공

## 13.3.1 텐서에서 텐서플로 데이터셋 만들기

`tf.data.Dataset.from_tensor_slices()` 함수  
`batch()` 매서드  

## 13.3.2 두 개의 텐서를 하나의 데이터셋으로 연결

In [21]:
tf.random.set_seed(1)
t_x = tf.random.uniform([4, 3], dtype=tf.float32)
t_y = tf.range(4)

In [22]:
# 특성이 있는 텐서와 클래스 레이블의 텐서 합치기
ds_joint = tf.data.Dataset.from_tensor_slices((t_x, t_y))
for example in ds_joint:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [0.165 0.901 0.631]  y: 0
 x: [0.435 0.292 0.643]  y: 1
 x: [0.976 0.435 0.66 ]  y: 2
 x: [0.605 0.637 0.614]  y: 3


In [23]:
# 원소 별 변환 하기
ds_trans = ds_joint.map(lambda x, y: (x * 2 - 1.0, y))
for example in ds_trans:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [-0.67   0.803  0.262]  y: 0
 x: [-0.131 -0.416  0.285]  y: 1
 x: [ 0.952 -0.13   0.32 ]  y: 2
 x: [0.21  0.273 0.229]  y: 3


### 13.3.3 shuffle(), batch(), repeat() 매서드

In [24]:
# 훈련 데이터를 무작위로 섞은 배치로 만들어 주입하려면
tf.random.set_seed(1)
ds = ds_joint.shuffle(buffer_size=len(t_x))  # t_x의 길이만큼 원소를 꺼내서 섞기
for example in ds:
    print(' x:', example[0].numpy(),
          ' y:', example[1].numpy())

 x: [0.976 0.435 0.66 ]  y: 2
 x: [0.435 0.292 0.643]  y: 1
 x: [0.165 0.901 0.631]  y: 0
 x: [0.605 0.637 0.614]  y: 3


In [25]:
# 배치로 만들기
ds = ds_joint.batch(batch_size=3, drop_remainder=False)
batch_x, batch_y = next(iter(ds))
print('배치 x:\n', batch_x.numpy())
print('배치 y: ', batch_y.numpy())

배치 x:
 [[0.165 0.901 0.631]
 [0.435 0.292 0.643]
 [0.976 0.435 0.66 ]]
배치 y:  [0 1 2]
